In [16]:
import numpy as np
import environment
import neural_controller

In [22]:
w = np.load("/Users/lorenzoleuzzi/Documents/GitHub/lifelong_evolutionary_swarms/ga_easy_200_best.npy")

In [18]:
len(w)

754

In [27]:
initial_setting = {
   'agents': np.array([[0, 5], [0, 10], [0, 15]], dtype=float),
   'blocks': np.array([[9, 16], [10, 12], [11, 6]], dtype=float),
   'colors': np.array([environment.RED, environment.GREEN, environment.RED], dtype=int)
}

env = environment.Environment(objective = [(environment.RED, environment.NORTH_EDGE)],
                   size = environment.SIMULATION_ARENA_SIZE, 
                   n_agents = 3, 
                   n_blocks = 3,
                   n_neighbors = 3,
                   sensor_range = environment.SIMULATION_SENSOR_RANGE,
                   sensor_angle = 360,
                   max_distance_covered_per_step = environment.SIMULATION_MAX_DISTANCE,
                   sensitivity = 0.2,
                   initial_setting = initial_setting)
env.reset(seed = 12)
env.print_env()

input_dim = (env.n_types + 2) * env.n_neighbors + env.n_types - 2
output_dim = 2
hidden_units = [16]
layer_sizes = [input_dim] + hidden_units + [output_dim]

nn = neural_controller.NeuralController(layer_sizes, hidden_activation="sigmoid", output_activation="sigmoid")
nn.summary()

. . . . . . 0 . . . . . . 1 . . . . . 2 . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . O . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . O . . . . . . . . .
. . . . . . . . O . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .


In [28]:
def calculate_fitness(individual, n_steps=100, verbose = False):
        
    fitness = 0
    obs, _ = env.reset()
    
    # Set the weights of the network
    nn.set_weights_from_vector(individual)

    for step in range(n_steps):
        
        nn_inputs = env.process_observation(obs)
        nn_outputs = np.array(nn.predict(nn_inputs))
        actions = np.round(nn_outputs * np.array([env.max_distance_covered_per_step, env.sensor_angle]), 1)
            
        obs, reward, done, _, _ = env.step(actions)

        fitness += reward

        if verbose:
            print("\nStep", step)
            # print("Observation: ", obs)
            # print("NN inputs: ", nn_inputs)
            print("Action: ", actions)
            env.print_env()
            print("Reward: ", reward)
        
        if done:
            fitness += (n_steps - step) / 2
            break
    
    return [float(fitness)]

In [29]:
calculate_fitness(w, verbose = True)


Step 0
Action:  [[1.5 5. ]
 [1.5 5. ]
 [1.5 5. ]]
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . 0 . . . . . . 1 . . . . . 2 . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . O . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . O . . . . . . . . .
. . . . . . . . O . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . .

[11.115186595291725]

In [30]:
nn.weights

[(array([[ 9.91003886e-01,  5.26133180e-01,  7.08612032e-01,
           9.37629376e-01, -5.72784183e-01, -7.24231914e-01,
           7.19439526e-01,  3.01341526e+00, -9.38467586e-01,
           9.69015933e-01,  2.83038030e-01,  1.61657161e+00,
          -5.46790883e-02,  1.96941400e-01, -1.90009405e-01,
          -6.62294070e-01],
         [-9.19307306e-01,  1.88006294e-02,  1.00982294e-01,
          -2.16158180e-01, -9.07981393e-01,  9.07227584e-02,
           2.18833060e+00, -1.28405901e+00,  9.01148658e-01,
          -5.51483313e-01, -8.92226460e-01,  6.17038232e-01,
          -1.03616319e-01, -6.95522304e-01,  7.31537593e-01,
           9.83570698e-01],
         [-4.50697541e-01, -2.63091214e-01,  5.19352593e-01,
          -3.84339854e-01, -6.55565716e-01, -5.89058273e-01,
           6.65609205e-01, -7.69046665e-01, -3.38594850e-01,
          -1.85024468e+00,  1.61072476e+00,  2.98194718e-01,
           2.96982068e+00,  7.93754860e-02, -2.80134293e-01,
          -2.07105757e-01],
 